In [3]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [13]:
# prepare our data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

num_samples, num_features = X.shape
print(num_samples, num_features) # 569 samples, 30 features

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# scale our features
sc = StandardScaler() # scales our features to have zero mean and unit variance
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# convert our data into torch tensors
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0], 1)


# the logistic regression model follows y = mx + b but then has a sigmoid at the end
class LogisticRegression(nn.Module):
  def __init__(self, num_input_features):
    super(LogisticRegression, self).__init__()
    self.Linear = nn.Linear(num_input_features, 1)
  
  def forward(self, x):
    y_pred = torch.sigmoid(self.Linear(x))
    return y_pred

model = LogisticRegression(num_features)


learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

num_epochs = 500

for epoch in range(num_epochs):
  # forward
  y_pred = model(X_train)
  loss = criterion(y_pred, y_train)

  # backward
  loss.backward()

  # update the weights
  optimizer.step()
  optimizer.zero_grad()

  if (epoch + 1) % 10 == 0:
    print(f"epoch: {epoch+1}, loss = {loss.item():.4f}")


# evaluate the model

with torch.no_grad():
  y_pred = model(X_test)
  # convert to class labels - since sigmoid returns prob.
  y_pred_classes = y_pred.round() # so our threshold is just 0.5
  acc = y_pred_classes.eq(y_test).sum() / float(y_test.shape[0])
  print(f'Accuracy = {acc:.4f}')


569 30
epoch: 10, loss = 0.6524
epoch: 20, loss = 0.5134
epoch: 30, loss = 0.4308
epoch: 40, loss = 0.3766
epoch: 50, loss = 0.3381
epoch: 60, loss = 0.3092
epoch: 70, loss = 0.2867
epoch: 80, loss = 0.2684
epoch: 90, loss = 0.2533
epoch: 100, loss = 0.2406
epoch: 110, loss = 0.2296
epoch: 120, loss = 0.2201
epoch: 130, loss = 0.2117
epoch: 140, loss = 0.2043
epoch: 150, loss = 0.1976
epoch: 160, loss = 0.1916
epoch: 170, loss = 0.1861
epoch: 180, loss = 0.1811
epoch: 190, loss = 0.1766
epoch: 200, loss = 0.1723
epoch: 210, loss = 0.1684
epoch: 220, loss = 0.1648
epoch: 230, loss = 0.1614
epoch: 240, loss = 0.1583
epoch: 250, loss = 0.1553
epoch: 260, loss = 0.1526
epoch: 270, loss = 0.1499
epoch: 280, loss = 0.1475
epoch: 290, loss = 0.1452
epoch: 300, loss = 0.1429
epoch: 310, loss = 0.1409
epoch: 320, loss = 0.1389
epoch: 330, loss = 0.1370
epoch: 340, loss = 0.1352
epoch: 350, loss = 0.1334
epoch: 360, loss = 0.1318
epoch: 370, loss = 0.1302
epoch: 380, loss = 0.1287
epoch: 390, lo